## Condiments

In [ ]:
import csv

# Transformed CSV file is in datasets folder - transformed_condiments_grouped.csv
file1 = input("Enter path to CSV file")

# Open the original CSV file
with open(file1, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    # Create empty lists for categories, years, and months
    categories = []
    years = []
    months = []

    # Loop through the rows of the original CSV file
    for row in csv_reader:
        # Add the category to the list of categories if it's not already in the list
        if row['Category'] not in categories:
            categories.append(row['Category'])

        # Add the year to the list of years if it's not already in the list
        year = int(row['Year'])
        if year not in years:
            years.append(year)

        # Add the month to the list of months if it's not already in the list
        if row['Month'] not in months:
            months.append(row['Month'])

    # Print the lists of categories, years, and months
    print('Categories:', categories)
    print('Years:', years)
    print('Months:', months)

n_months = len(months)
n_categories = len(categories)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from dash import html
from dash import dcc

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv(file1)

app = JupyterDash(__name__)


# define the app layout
app.layout = html.Div([
    html.H1("CycloTrend", style={'text-align': 'center'}),
    html.Div([
        html.Div([
            dcc.Slider(
                id='year-slider',
                min=min(years),
                max=max(years),
                step=1,
                value=min(years),
                marks={str(year): str(year) for year in years}
            ),
            dcc.Interval(
                id='slider-interval',
                interval=3000,  # update every 3000ms (1 second)
                n_intervals=0
            ),
        html.Button(id='pause-button', children='Pause', n_clicks=0,style={
                        'font-size': '15px',
                        'padding': '4px',
                        'margin-bottom': '5px',
                        'display': 'inline-block'
                    })
        ], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Checklist(
                id='line-toggle',
                options=[{'label': 'Hover for polar linear plot', 'value': 'line'}],
                value=[],
                style={
                        'font-size': '20px',
                        'padding': '8px',
                        'margin-bottom': '10px',
                        'display': 'inline-block'
                    }
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    dcc.Graph(
        id='polar-chart',
        style={'width': '800px', 'height': '700px', 'margin': 'auto'}
    ),
    dcc.Store(id='hover-store', data=False)
])


category_colors = {'Mayonnaise': 'dodgerblue', 'Ketchup': 'red', 'Mustard': 'gold', 'Soy sauce': 'seagreen', 'Barbecue sauce' : 'mediumpurple'}


# define the callback function for the pause button
@app.callback(
    Output('slider-interval', 'disabled'),
    Output('pause-button', 'children'),
    Input('pause-button', 'n_clicks'),
    State('slider-interval', 'disabled')
)
def pause_slider(n_clicks, disabled):
    if n_clicks % 2 == 1:
        return True, 'Resume'
    else:
        return False, 'Pause'

def get_tooltip_text(data, column, label):
    tooltip_text = []
    for i in range(len(data)):
        if i == 0:
            tooltip_text.append(f"{label}: {data.iloc[i][column]}")
        else:
            change = data.iloc[i][column] - data.iloc[i-1][column]
            if change > 0:
                tooltip_text.append(f"{label}: {data.iloc[i][column]} (+{change})")
            elif change < 0:
                tooltip_text.append(f"{label}: {data.iloc[i][column]} ({change})")
            else:
                tooltip_text.append(f"{label}: {data.iloc[i][column]}")
    return tooltip_text




@app.callback(
    Output('hover-store', 'data'),
    Input('line-toggle', 'value')
)

def toggle_hover(value):
    return 'line' in value


@app.callback(
    Output('year-slider', 'value'),
    Input('slider-interval', 'n_intervals'),
    State('year-slider', 'value'),
    prevent_initial_call=True
)

def update_slider(n_intervals, value):
    if value == max(years):
        return min(years)
    else:
        return value + 1


@app.callback(
    Output('polar-chart', 'figure'),
    Input('year-slider', 'value'),
    Input('hover-store', 'data'),
    Input('polar-chart', 'hoverData')
)


def update_polar_chart(year, is_hover_enabled, hoverData):
    if not is_hover_enabled:
        hoverData = None

    filtered_df = df[df['Year'] == year]

    # default to scatter plot
    polar_type = 'scatter'

    if hoverData is not None:
        selected_df = None  # initialize the variable with a default value
        if hoverData['points'][0]['curveNumber'] != -1:
            selected_cat = hoverData['points'][0]['curveNumber']
            if selected_cat < len(categories):
                selected_df = filtered_df[filtered_df['Category'] == categories[selected_cat]]
                polar_type = 'line'

    # create a copy of the filtered dataframe and calculate the change in value compared to the previous year
    prev_year = year - 1
    prev_year_df = df[df['Year'] == prev_year].copy()
    prev_year_df.rename(columns={'Value': 'Prev_Value'}, inplace=True)
    prev_year_df.drop('Year', axis=1, inplace=True)
    filtered_df = pd.merge(filtered_df, prev_year_df, on=['Month', 'Category'], how='left')
    filtered_df['Change'] = filtered_df['Value'] - filtered_df['Prev_Value']

    fig = px.scatter_polar(filtered_df, r='Value', theta='Month', color='Category',
                           color_discrete_map=category_colors, hover_data=['Change'])

    if polar_type == 'line':
        if hoverData['points'][0]['curveNumber'] != -1:
            # add the polar line plot trace to the scatter plot trace
            selected_color = category_colors[categories[selected_cat]]
            fig.add_trace(px.line_polar(selected_df, r='Value', theta='Month',
                                        color_discrete_sequence=[selected_color]).data[0])

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                showticklabels=False,
                showgrid=False
            )
        ),
        #width=700,
        #height=700,
        #olar_bgcolor='black'
    )

    fig.update_traces(
        marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey')),
        selector=dict(mode='markers'),
        hovertemplate='Month: %{theta}<br>' +
                      'Value: %{r:.0f}<br>' +
                      'Change from previous year: %{customdata[0]:+.0f}'
    )

    if hoverData is None:
        # reset the chart to the default state (scatter plot)
        fig.update_traces(mode='markers')

    return fig


# run the app
app.run_server(port=8051)

## Late night TV hosts 

In [1]:
import csv


# Transformed CSV file is in datasets folder - transformed_latenighthosts_grouped.csv
file2 = input("Enter path to CSV file")

# Open the original CSV file
with open(file2, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    # Create empty lists for categories, years, and months
    categories = []
    years = []
    months = []

    # Loop through the rows of the original CSV file
    for row in csv_reader:
        # Add the category to the list of categories if it's not already in the list
        if row['Category'] not in categories:
            categories.append(row['Category'])

        # Add the year to the list of years if it's not already in the list
        year = int(row['Year'])
        if year not in years:
            years.append(year)

        # Add the month to the list of months if it's not already in the list
        if row['Month'] not in months:
            months.append(row['Month'])

    # Print the lists of categories, years, and months
    print('Categories:', categories)
    print('Years:', years)
    print('Months:', months)

n_months = len(months)
n_categories = len(categories)

Enter path to CSV file/Users/mihirchhatre/Documents/NYU/Semester 2/DataViz/CycloTrends/Datasets/transformed_latenighthosts_grouped.csv
Categories: ['Jimmy Fallon', 'Seth Meyers', 'Stephen Colbert', 'Trevor Noah', 'James Corden']
Years: [2018, 2019, 2020, 2021, 2022]
Months: ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from dash import html
from dash import dcc

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv(file2)

app = JupyterDash(__name__)


# define the app layout
app.layout = html.Div([
    html.H1("Cyclotrend", style={'text-align': 'center'}),
    html.Div([
        html.Div([
            dcc.Slider(
                id='year-slider',
                min=min(years),
                max=max(years),
                step=1,
                value=min(years),
                marks={str(year): str(year) for year in years}
            ),
            dcc.Interval(
                id='slider-interval',
                interval=3000,  # update every 3000ms (1 second)
                n_intervals=0
            ),
        html.Button(id='pause-button', children='Pause', n_clicks=0,style={
                        'font-size': '15px',
                        'padding': '4px',
                        'margin-bottom': '5px',
                        'display': 'inline-block'
                    })
        ], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Checklist(
                id='line-toggle',
                options=[{'label': 'Hover for polar linear plot', 'value': 'line'}],
                value=[],
                style={
                        'font-size': '20px',
                        'padding': '8px',
                        'margin-bottom': '10px',
                        'display': 'inline-block'
                    }
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    dcc.Graph(
        id='polar-chart',
        style={'width': '800px', 'height': '700px', 'margin': 'auto'}
    ),
    dcc.Store(id='hover-store', data=False)
])


category_colors = {'Jimmy Fallon': 'dodgerblue', 'Seth Meyers': 'red', 'Stephen Colbert': 'gold', 'Trevor Noah': 'seagreen', 'James Corden' : 'mediumpurple'}


# define the callback function for the pause button
@app.callback(
    Output('slider-interval', 'disabled'),
    Output('pause-button', 'children'),
    Input('pause-button', 'n_clicks'),
    State('slider-interval', 'disabled')
)
def pause_slider(n_clicks, disabled):
    if n_clicks % 2 == 1:
        return True, 'Resume'
    else:
        return False, 'Pause'

def get_tooltip_text(data, column, label):
    tooltip_text = []
    for i in range(len(data)):
        if i == 0:
            tooltip_text.append(f"{label}: {data.iloc[i][column]}")
        else:
            change = data.iloc[i][column] - data.iloc[i-1][column]
            if change > 0:
                tooltip_text.append(f"{label}: {data.iloc[i][column]} (+{change})")
            elif change < 0:
                tooltip_text.append(f"{label}: {data.iloc[i][column]} ({change})")
            else:
                tooltip_text.append(f"{label}: {data.iloc[i][column]}")
    return tooltip_text




@app.callback(
    Output('hover-store', 'data'),
    Input('line-toggle', 'value')
)

def toggle_hover(value):
    return 'line' in value


@app.callback(
    Output('year-slider', 'value'),
    Input('slider-interval', 'n_intervals'),
    State('year-slider', 'value'),
    prevent_initial_call=True
)

def update_slider(n_intervals, value):
    if value == max(years):
        return min(years)
    else:
        return value + 1


@app.callback(
    Output('polar-chart', 'figure'),
    Input('year-slider', 'value'),
    Input('hover-store', 'data'),
    Input('polar-chart', 'hoverData')
)


def update_polar_chart(year, is_hover_enabled, hoverData):
    if not is_hover_enabled:
        hoverData = None

    filtered_df = df[df['Year'] == year]

    # default to scatter plot
    polar_type = 'scatter'

    if hoverData is not None:
        selected_df = None  # initialize the variable with a default value
        if hoverData['points'][0]['curveNumber'] != -1:
            selected_cat = hoverData['points'][0]['curveNumber']
            if selected_cat < len(categories):
                selected_df = filtered_df[filtered_df['Category'] == categories[selected_cat]]
                polar_type = 'line'

    # create a copy of the filtered dataframe and calculate the change in value compared to the previous year
    prev_year = year - 1
    prev_year_df = df[df['Year'] == prev_year].copy()
    prev_year_df.rename(columns={'Value': 'Prev_Value'}, inplace=True)
    prev_year_df.drop('Year', axis=1, inplace=True)
    filtered_df = pd.merge(filtered_df, prev_year_df, on=['Month', 'Category'], how='left')
    filtered_df['Change'] = filtered_df['Value'] - filtered_df['Prev_Value']

    fig = px.scatter_polar(filtered_df, r='Value', theta='Month', color='Category',
                           color_discrete_map=category_colors, hover_data=['Change'])

    if polar_type == 'line':
        if hoverData['points'][0]['curveNumber'] != -1:
            # add the polar line plot trace to the scatter plot trace
            selected_color = category_colors[categories[selected_cat]]
            fig.add_trace(px.line_polar(selected_df, r='Value', theta='Month',
                                        color_discrete_sequence=[selected_color]).data[0])

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                showticklabels=False,
                showgrid=False
            )
        ),
        #width=700,
        #height=700,
        #olar_bgcolor='black'
    )

    fig.update_traces(
        marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey')),
        selector=dict(mode='markers'),
        hovertemplate='Month: %{theta}<br>' +
                      'Value: %{r:.0f}<br>' +
                      'Change from previous year: %{customdata[0]:+.0f}'
    )

    if hoverData is None:
        # reset the chart to the default state (scatter plot)
        fig.update_traces(mode='markers')

    return fig


# run the app
app.run_server(port=8050)

Dash app running on http://127.0.0.1:8050/
